In [2]:
from sympy import *
init_printing()

In [3]:
def P_op(In_size, n, m):
    return eye(In_size).elementary_row_op('n<->m', n, m)

def M_op(In_size, n, k):
    return eye(In_size).elementary_row_op('n->kn', n, k)

def A_op(In_size, n, m, k):
    return eye(In_size).elementary_row_op('n->n+km', row1=n, row2=m, k=k)

In [4]:
def smart_rref_decomposition(rref):
    elementary_matrices = []
    # количество нулевых строк в rref
    null_row_count = 0
    row = 0
    # вычеркиваем нулевые строки
    while row < rref.shape[0]:
        if rref.row(row) == zeros(1, rref.shape[1]):
            rref.row_del(row)
        row += 1
    # индексы ячеек главной диагонали
    diag_i, diag_j = 0, 0
    while diag_i < rref.shape[0] and diag_j < rref.shape[1]:
        element_to_one = rref[diag_i, diag_j]
        if element_to_one == 0:
            # ищем ненулевой элемент в столбце
            found = False
            for row in range(diag_i + 1, rref.shape[0]):
                # если нашли
                if rref[row, diag_j] != 0:
                    element_to_one = rref[row, diag_j]
                    # перестановка
                    op = P_op(rref.shape[0], diag_i, row)
                    # сохраняем перестановку
                    elementary_matrices.append(op)
                    rref = op * rref
                    found = True
                    break
            # если не нашли
            if found == False:
                # смещаемся вправо
                diag_j += 1
                continue  
        # приводим к единице, если надо
        if element_to_one != 1: 
            op = M_op(rref.shape[0], diag_i, 1 / element_to_one)
            elementary_matrices.append(op)
            rref = op * rref   
        # приводим к нулю элементы под и над данной ячейкой
        row = 0 
        while row < rref.shape[0]:
            if row != diag_i and rref[row, diag_j] != 0 and rref.row(row) != zeros(1, rref.shape[1]):
                element_to_zero = rref[row, diag_j]
                op = A_op(rref.shape[0], row, diag_i, -element_to_zero)
                elementary_matrices.append(op)
                rref = op * rref  
                # если получили нулевую строку,
                # то вычеркиваем ее
                if rref.row(row) == zeros(1, rref.shape[1]):
                    rref.row_del(row)  
                    row -= 1
                    if (row < diag_i):
                        diag_i -= 1
                    # и запоминаем, что вычеркнули
                    null_row_count += 1
            row += 1         
        diag_i += 1
        diag_j += 1            
    # записываем обратно вычеркнутые строки
    for i in range(null_row_count):
        rref = rref.row_insert(rref.shape[0], zeros(1, rref.shape[1]))
    return rref, elementary_matrices

In [5]:
# разложение невырожденных матриц на произведение элементарных
def nondegenerate_decomposition(A):
    rref, elementary_matrices = smart_rref_decomposition(A)
    elementary_matrices.append(rref)
    elementary_matrices_reversed = list(reversed(elementary_matrices))
    elementary_matrices_reversed_inv = [i.inv() for i in elementary_matrices_reversed]
    return elementary_matrices_reversed_inv

In [6]:
def smart_snf_decomposition(A):
    m, n = A.shape
    A_Im = Matrix.hstack(A, eye(m))
    U = A_Im.rref()[0][:, n:]
    R = A.rref()[0]
    RT_In = Matrix.hstack(R.T, eye(n))
    VT = RT_In.rref()[0][:, m:]
    V = VT.T
    return U*A*V, nondegenerate_decomposition(U), nondegenerate_decomposition(V)


In [9]:
A = Matrix([
    [1,-1,1,2],
    [2,-2,1,-1],
    [-1,1,0,3]
])
SNF, U_decomposition, V_decomposition = smart_snf_decomposition(A)
SNF, U_decomposition, V_decomposition

⎛                                                                             
⎜⎡1  0  0  0⎤  ⎡⎡1  0  0⎤  ⎡1  0  0⎤  ⎡1  0  1⎤  ⎡1  0  0 ⎤  ⎡1  -1  0⎤  ⎡0  0
⎜⎢          ⎥  ⎢⎢       ⎥  ⎢       ⎥  ⎢       ⎥  ⎢        ⎥  ⎢        ⎥  ⎢    
⎜⎢0  1  0  0⎥, ⎢⎢0  1  0⎥, ⎢0  1  2⎥, ⎢0  1  0⎥, ⎢0  1  0 ⎥, ⎢0  1   0⎥, ⎢0  1
⎜⎢          ⎥  ⎢⎢       ⎥  ⎢       ⎥  ⎢       ⎥  ⎢        ⎥  ⎢        ⎥  ⎢    
⎜⎣0  0  0  0⎦  ⎣⎣0  0  1⎦  ⎣0  0  1⎦  ⎣0  0  1⎦  ⎣0  0  -1⎦  ⎣0  0   1⎦  ⎣1  0
⎝                                                                             

       ⎡⎡1  0  0  0⎤  ⎡1  0  0  1⎤  ⎡1  0  -1  0⎤  ⎡1  3/5  0  0⎤  ⎡1   0   0 
  1⎤⎤  ⎢⎢          ⎥  ⎢          ⎥  ⎢           ⎥  ⎢            ⎥  ⎢          
   ⎥⎥  ⎢⎢0  1  0  0⎥  ⎢0  1  0  0⎥  ⎢0  1  0   0⎥  ⎢0   1   0  0⎥  ⎢0  1/5  0 
  0⎥⎥, ⎢⎢          ⎥, ⎢          ⎥, ⎢           ⎥, ⎢            ⎥, ⎢          
   ⎥⎥  ⎢⎢0  0  1  0⎥  ⎢0  0  1  0⎥  ⎢0  0  1   0⎥  ⎢0   0   1  0⎥  ⎢0   0   1 
  0⎦⎦  ⎢⎢          ⎥  ⎢          ⎥  ⎢           ⎥  

In [10]:
U = 1
for e_inv in U_decomposition:
    U = e_inv * U
V = 1
for e_inv in V_decomposition:
    V = e_inv * V
U, V, SNF, U*A*V

⎛             ⎡ 0    0   1     0  ⎤                            ⎞
⎜⎡0  0   -1⎤  ⎢                   ⎥  ⎡1  0  0  0⎤  ⎡1  0  0  0⎤⎟
⎜⎢         ⎥  ⎢ 0    0   0     1  ⎥  ⎢          ⎥  ⎢          ⎥⎟
⎜⎢0  1   2 ⎥, ⎢                   ⎥, ⎢0  1  0  0⎥, ⎢0  1  0  0⎥⎟
⎜⎢         ⎥  ⎢5/3   1  -5/3  5/3 ⎥  ⎢          ⎥  ⎢          ⎥⎟
⎜⎣1  -1  -1⎦  ⎢                   ⎥  ⎣0  0  0  0⎦  ⎣0  0  0  0⎦⎟
⎝             ⎣-1/3  0  1/3   -1/3⎦                            ⎠